In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [5]:
# 假设 df 是包含所有点位的特征数据的 DataFrame
df = pd.read_csv(r"D:\ArcGISProjects\workspace\frequency_filter\dy_frequency.csv")
df.fillna(df.select_dtypes(include='number').mean(),inplace=True)

In [6]:
df.columns

Index(['TL', 'YL', 'TS', 'TZ', 'DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature',
       'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity',
       'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE',
       'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT'],
      dtype='object')

In [27]:



# 分组的环境因子
env_factors = ['DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature',
       'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity',
       'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE',
       'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT']

# 基于土壤类型进行分组
soil_types = df['TL'].unique()
representative_samples = []

for soil_type in soil_types:
    # 提取当前土壤类型的样本，并显式复制数据以避免警告
    soil_type_df = df[df['TL'] == soil_type].copy()

    # 聚类分析
    kmeans = KMeans(n_clusters=1)
    soil_type_df.loc[:, 'cluster'] = kmeans.fit_predict(soil_type_df[env_factors])

    # 计算每个样本到聚类中心的距离
    center = kmeans.cluster_centers_[0]
    distances = np.linalg.norm(soil_type_df[env_factors].values - center, axis=1)

    # 设置距离阈值
    threshold = np.percentile(distances, 20)  # 例如选择距离在前75%的样本

    # 保留距离小于或等于阈值的样本
    representative_samples.extend(soil_type_df[distances <= threshold].to_dict('records'))

representative_df = pd.DataFrame(representative_samples)



In [28]:
len(representative_df)

5063

In [29]:
representative_df['TL'].value_counts()

黄壤         2928
None        885
水稻土         642
石灰土         597
紫色土           6
山地灌丛草甸土       5
Name: TL, dtype: int64